In [10]:
%load_ext autoreload
%autoreload 2
from finetune import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
args

{'cultural_corpus': 'yo-bm25-10000',
 'pretrained_model': 'FacebookAI/xlm-roberta-base',
 'corpus_chunk_size': 128,
 'wwm_probability': 0.15,
 'batch_size': 64}

In [3]:
tokenizer = mk_tokenizer(args)
mlm_model, mc_model = mk_models(args)

mask token id is 250001 and mask token is <mask>
tokenizer max len 512
model config: XLMRobertaConfig {
  "_name_or_path": "FacebookAI/xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}



Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForMultipleChoice were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mlm_model num param: 278295186
mc_model num param: 278044417


In [4]:
fill_mask(args, mlm_model, tokenizer, f"The cat said {tokenizer.decode(tokenizer.mask_token_id)}.")

input: The cat said <mask>.
pred 0: The cat said _:_.
pred 1: The cat said _,_.
pred 2: The cat said _that_.
pred 3: The cat said _:_.
pred 4: The cat said _._.
pred 5: The cat said _it_.
pred 6: The cat said _he_.
pred 7: The cat said _"_.
pred 8: The cat said _to_.
pred 9: The cat said _..._.


In [5]:
toy_text = f"The cat said {tokenizer.decode(tokenizer.mask_token_id)}."
toy_input = tokenizer(toy_text, return_tensors="pt")
mlm_model(**toy_input).keys()

odict_keys(['logits', 'hidden_states'])

In [23]:
lm_corpus, corpus_mask_collator, corpus_wwm_collator = mk_corpus(args, tokenizer)
lm_corpus

Map: 100%|██████████| 30/30 [00:00<00:00, 10412.87 examples/s]

index 0 train example:
{'input_ids': [0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 581, 7515, 2804, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 18826, 7, 5154, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 18826, 7, 5154, 163, 8770, 2, 0, 581, 7515, 2804, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 18826, 7, 5154, 163, 8770, 2, 0, 581, 7515, 2804, 163, 8770, 2, 0, 581, 7515, 2804, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 18826, 7, 5154, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 717], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 16
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1
    })
})

In [24]:
processed_figqa_datasets, figqa_data_collator = mk_figqa_dataset(args, tokenizer)
processed_figqa_datasets

Map: 100%|██████████| 1094/1094 [00:00<00:00, 22470.51 examples/s]

figqa data looks like:
train: {'labels': 0, 'input_ids': [[0, 1840, 2565, 1902, 70, 90254, 111, 140147, 6664, 5, 2, 2, 1840, 103036, 7, 831, 186, 18822, 71, 5, 2], [0, 1840, 2565, 1902, 70, 90254, 111, 140147, 6664, 5, 2, 2, 1840, 103036, 7, 53418, 186, 63207, 297, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
decoded: ['<s> Her word had the strength of titanium.</s></s> Her promises can be believed.</s>', '<s> Her word had the strength of titanium.</s></s> Her promises cannot be trusted.</s>']
dev: {'labels': 0, 'input_ids': [[0, 581, 23040, 1902, 70, 113014, 12741, 7, 111, 10, 27148, 15555, 2, 2, 581, 23040, 509, 4552, 11348, 133, 5, 2], [0, 581, 23040, 1902, 70, 113014, 12741, 7, 111, 10, 27148, 15555, 2, 2, 581, 23040, 509, 4552, 144142, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1458
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1094
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 600
    })
})

## MLM train

In [8]:
training_args = TrainingArguments(
    output_dir=f"xlmr-finetuned",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=5e-4,
    weight_decay=0.01,
    per_device_train_batch_size=args['batch_size'],
    per_device_eval_batch_size=args['batch_size'],
    logging_steps=1,
    num_train_epochs=10,
    use_cpu=True,
    use_mps_device=False,
)
trainer = Trainer(
    model=mlm_model,
    args=training_args,
    train_dataset=lm_corpus["train"],
    eval_dataset=lm_corpus["val"],
    data_collator=corpus_mask_collator,
    tokenizer=tokenizer,
)

/Users/chaosarium/anaconda3/envs/multi/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [9]:
trainer.evaluate()

100%|██████████| 1/1 [00:00<00:00, 5229.81it/s]


{'eval_loss': 12.164128303527832,
 'eval_runtime': 0.2334,
 'eval_samples_per_second': 4.284,
 'eval_steps_per_second': 4.284}

In [ ]:
trainer.train()

## FigQA training

In [26]:
train_dataset = processed_figqa_datasets["train"]
eval_dataset = processed_figqa_datasets["validation"]

train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=figqa_data_collator, batch_size=args['batch_size'])
eval_dataloader = DataLoader(eval_dataset, collate_fn=figqa_data_collator, batch_size=args['batch_size'])

In [28]:
optimizer = torch.optim.AdamW(mc_model.parameters(), lr=2e-5)
lr_scheduler = get_scheduler(
    name='linear',
    optimizer=optimizer,
    num_warmup_steps=3,
    num_training_steps=30,
)

In [33]:
mc_model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(mc_model, optimizer, train_dataloader, eval_dataloader, lr_scheduler)

In [ ]:
completed_steps = 0

for epoch in range(0, 10):
    model.train()
    model, total_loss, completed_steps = train_model(train_dataloader, model, accelerator, optimizer, lr_scheduler, args, completed_steps, checkpointing_steps, progress_bar)

    eval_metric = eval_model(model, eval_dataloader, metric, accelerator, epoch, args)

    if args["with_tracking"]:
        accelerator.log(
            {
                "accuracy": eval_metric,
                "train_loss": total_loss.item() / len(train_dataloader),
                "epoch": epoch,
                "step": completed_steps,
            },
            step=completed_steps,
        )

    if args["push_to_hub"] and epoch < args["num_train_epochs"] - 1:
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(
            args["output_dir"], is_main_process=accelerator.is_main_process, save_function=accelerator.save
        )
        if accelerator.is_main_process:
            tokenizer.save_pretrained(args["output_dir"])
            # repo.push_to_hub(
            #     commit_message=f"Training in progress epoch {epoch}", blocking=False, auto_lfs_prune=True
            # )

    if args["checkpointing_steps"] == "epoch":
        output_dir = f"epoch_{epoch}"
        if args["output_dir"] is not None:
            output_dir = os.path.join(args["output_dir"], output_dir)
        accelerator.save_state(output_dir)

if args["output_dir"] is not None:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(
        args["output_dir"], is_main_process=accelerator.is_main_process, save_function=accelerator.save
    )
    if accelerator.is_main_process:
        tokenizer.save_pretrained(args["output_dir"])
        # if args["push_to_hub"]:
        #     repo.push_to_hub(commit_message="End of training", auto_lfs_prune=True)
    with open(os.path.join(args["output_dir"], "all_results.json"), "w") as f:
        json.dump({"eval_accuracy": eval_metric["accuracy"]}, f)

eval_metric["accuracy"]
